# Fase 3 - Densificação Híbrida (Vetorial + Agentes)

Nesta etapa, utilizamos uma abordagem agressiva, porém controlada, para densificar o grafo de conhecimento.

**Diagnóstico:** A densidade inicial (~0.004) é insuficiente para algoritmos de Knowledge Tracing (SINKT), que dependem de caminhos conectados para propagar inferência.

**Pipeline Atualizado:**
1.  **The Cleaner (Faxineiro):** Remove ruídos de extração antes do processamento caro.
2.  **The Architect (Híbrido):**
    *   **Matemática (Scout):** Gera embeddings (text-embedding-3-small) e usa similaridade de cosseno com threshold rígido (0.89) para encontrar candidatos a conexão.
    *   **LLM (Validator):** Valida semanticamente os candidatos matemáticos e define o tipo de aresta (USE, RELATED_TO).
3.  **The Teacher (Pedagogo):** Analisa as novas conexões e promove para PREREQUISITE quando há dependência de aprendizado.

---

In [1]:
!pip install -q langchain langchain-openai networkx scikit-learn numpy pydantic tqdm python-dotenv

import os
import json
import numpy as np
import networkx as nx
from typing import List, Literal, Tuple
from tqdm import tqdm
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

# Configuração de Modelos
llm_mini = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# --- NOVO DIRETÓRIO DE ENTRADA/SAÍDA ---
INPUT_FOLDER = "output/01_extraction"
OUTPUT_FOLDER = "output/02_densification"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

CONCEPTS_FILE = f"{INPUT_FOLDER}/concepts_map.json"
RELATIONS_FILE = f"{INPUT_FOLDER}/relations_initial.json" # Atualizado para ler o novo output da fase 2
ENHANCED_FILE = f"{OUTPUT_FOLDER}/enhanced_graph.json"

In [2]:
## 1. Carregamento e Estado Inicial

def load_state():
    with open(CONCEPTS_FILE, 'r', encoding='utf-8') as f:
        concepts = json.load(f)
    with open(RELATIONS_FILE, 'r', encoding='utf-8') as f:
        relations = json.load(f)
    return concepts, relations

concepts, relations = load_state()

# Construir Grafo Inicial para verificação de existência
G_initial = nx.DiGraph()
for r in relations:
    G_initial.add_edge(r['source'], r['target'])

print(f"Estado Inicial:")
print(f"Nodes: {len(concepts)} | Edges: {len(relations)}")
print(f"Densidade: {nx.density(G_initial):.5f}")

Estado Inicial:
Nodes: 230 | Edges: 211
Densidade: 0.00585


**Agente de Limpeza (Cleaner):** Executa uma varredura inicial para remover conceitos que são claramente erros de extração ou irrelevantes, economizando tokens e processamento nas etapas seguintes.

In [3]:
## 2. Agente 1: The Cleaner (Faxineiro)
# Removemos ruídos antes de gerar embeddings para não sujar o espaço vetorial.

class NoiseConcept(BaseModel):
    nome: str = Field(description="Nome do conceito ruído")
    razao: str

class ConceptReview(BaseModel):
    noise_concepts: List[NoiseConcept]

cleaner_chain = (
    ChatPromptTemplate.from_messages([
        ("system", """Identifique ruídos na lista de conceitos técnicos de Linux.
        RUÍDOS: Variáveis soltas ($VAR), erros de OCR, números, termos do livro (Capítulo, Página), verbos soltos.
        MANTENHA: Comandos, caminhos (/etc), siglas, conceitos teóricos."""),
        ("user", "Lista: {concepts}\n{format_instructions}")
    ])
    | llm_mini
    | PydanticOutputParser(pydantic_object=ConceptReview)
)

def run_cleaner(all_concepts):
    names = [c['nome'] for c in all_concepts]
    noise_found = []
    batch_size = 150
    
    print("\n🧹 Cleaner trabalhando...")
    for i in range(0, len(names), batch_size):
        batch = names[i:i+batch_size]
        try:
            res = cleaner_chain.invoke({
                "concepts": json.dumps(batch, ensure_ascii=False),
                "format_instructions": PydanticOutputParser(pydantic_object=ConceptReview).get_format_instructions()
            })
            if res.noise_concepts:
                noise_found.extend([n.nome for n in res.noise_concepts])
                print(f"   Batch {i//batch_size + 1}: {len(res.noise_concepts)} ruídos encontrados.")
        except Exception as e:
            print(f"Erro Cleaner: {e}")
            
    return set(noise_found)

noise_set = run_cleaner(concepts)
clean_concepts = [c for c in concepts if c['nome'] not in noise_set]
print(f"Conceitos Válidos: {len(clean_concepts)} (Ruídos removidos: {len(noise_set)})")


🧹 Cleaner trabalhando...
   Batch 1: 17 ruídos encontrados.
   Batch 2: 9 ruídos encontrados.
Conceitos Válidos: 204 (Ruídos removidos: 26)


**Geração de Embeddings:** Cria representações vetoriais ricas para cada conceito, combinando nome, tipo e definição. Isso permite encontrar similaridades semânticas profundas que não seriam detectadas apenas por comparação de strings.

In [4]:
## 3. Preparação Vetorial (Embeddings)

print("\n🧬 Gerando Embeddings (text-embedding-3-small)...")

# Criar representação rica para o embedding: "Nome (Tipo): Definição"
texts_to_embed = [f"{c['nome']} ({c['tipo']}): {c['definicao']}" for c in clean_concepts]

try:
    vectors = embeddings_model.embed_documents(texts_to_embed)
    vectors_np = np.array(vectors)
    print(f"Embeddings gerados. Shape: {vectors_np.shape}")
except Exception as e:
    print(f"Erro ao gerar embeddings: {e}")
    vectors_np = None


🧬 Gerando Embeddings (text-embedding-3-small)...
Embeddings gerados. Shape: (204, 1536)


**Busca Vetorial e Filtragem:** Calcula a matriz de similaridade entre todos os conceitos e aplica regras rígidas (threshold > 0.89, top-k=5) para selecionar pares candidatos a conexão, evitando a criação de 'super-hubs' ou conexões fracas.

In [5]:
## 4. Algoritmo de Densificação (Matemática)

# --- AJUSTE ESTRATÉGICO: Diminuição do Threshold ---
# Antes: 0.89 (Muito estrito) -> Agora: 0.82 (Permissivo, o Agente Juiz da Fase 4 limpará)
SIMILARITY_THRESHOLD = 0.82 
TOP_K = 8 # Aumentado de 5 para 8 para capturar mais vizinhos potenciais

candidates = [] # Lista de tuplas (idx_A, idx_B, score)

if vectors_np is not None:
    print("\nCalculando Matriz de Similaridade...")
    sim_matrix = cosine_similarity(vectors_np)
    
    count_candidates = 0
    
    # Iterar sobre cada nó para encontrar vizinhos
    for i in range(len(clean_concepts)):
        # Pegar scores para o nó i
        scores = sim_matrix[i]
        
        # Zerar o score dele mesmo para não ser selecionado
        scores[i] = 0
        
        # Filtrar por Threshold
        # Retorna índices onde score > threshold
        high_sim_indices = np.where(scores > SIMILARITY_THRESHOLD)[0]
        
        # Ordenar por score decrescente e pegar Top-K
        # (Precisamos ordenar os índices filtrados baseados em seus scores)
        top_indices = sorted(high_sim_indices, key=lambda idx: scores[idx], reverse=True)[:TOP_K]
        
        source_name = clean_concepts[i]['nome']
        
        for j in top_indices:
            target_name = clean_concepts[j]['nome']
            
            # Verificar se já existe conexão (Direção A->B ou B->A)
            # Queremos evitar redundância se o grafo for não-direcionado semanticamente,
            # mas aqui é direcionado. Porém, se já existe A->B, não sugerimos de novo.
            if not G_initial.has_edge(source_name, target_name):
                candidates.append({
                    "source": clean_concepts[i],
                    "target": clean_concepts[j],
                    "score": float(scores[j])
                })
                count_candidates += 1

    print(f"Candidatos Matemáticos Encontrados: {len(candidates)}")
    print(f"   (Critérios: Sim > {SIMILARITY_THRESHOLD}, Top-{TOP_K}, Sem conexões prévias)")


Calculando Matriz de Similaridade...
Candidatos Matemáticos Encontrados: 16
   (Critérios: Sim > 0.82, Top-8, Sem conexões prévias)


**Validação Semântica (LLM):** Submete os candidatos encontrados matematicamente à análise do GPT-4o-mini. O modelo decide se a relação faz sentido tecnicamente e atribui o tipo correto (USE, RELATED_TO) ou descarta o par (SKIP).

In [6]:
## 5. Agente 2: The Architect (Validação LLM)

class ValidatedEdge(BaseModel):
    source: str
    target: str
    relation_type: Literal['RELATED_TO', 'USE', 'SKIP']
    explanation: str

class ValidationOutput(BaseModel):
    edges: List[ValidatedEdge]

architect_chain = (
    ChatPromptTemplate.from_messages([
        ("system", """Você é um Arquiteto de Ontologias Linux.
        Receba pares de conceitos com alta similaridade vetorial.
        Sua tarefa: Validar se existe relação lógica e tipificá-la.

        REGRAS:
        1. Se forem Sinônimos ou Variantes (ex: 'ls' e 'listar'): Use 'RELATED_TO'.
        2. Se um é ferramenta/comando do outro (ex: 'apt' e 'Gerenciador de Pacotes'): Use 'USE'.
        3. Se for apenas coincidência de palavras sem relação técnica direta: Use 'SKIP'.
        """),
        ("user", """Analise estes candidatos:
        {candidates}
        
        {format_instructions}""")
    ])
    | llm_mini
    | PydanticOutputParser(pydantic_object=ValidationOutput)
)

validated_edges = []

if candidates:
    print("\n Arquiteto Validando Candidatos...")
    
    # Batching para LLM
    BATCH_SIZE = 20
    
    for i in range(0, len(candidates), BATCH_SIZE):
        batch = candidates[i:i+BATCH_SIZE]
        
        # Formatar input compacta para economizar tokens
        batch_input = []
        for item in batch:
            s = item['source']
            t = item['target']
            batch_input.append(f"- {s['nome']} ({s['tipo']}) <--> {t['nome']} ({t['tipo']}) [Sim: {item['score']:.2f}]")
            
        try:
            res = architect_chain.invoke({
                "candidates": "\n".join(batch_input),
                "format_instructions": PydanticOutputParser(pydantic_object=ValidationOutput).get_format_instructions()
            })
            
            valid_ones = [e for e in res.edges if e.relation_type != 'SKIP']
            validated_edges.extend(valid_ones)
            print(f"   Batch {i//BATCH_SIZE + 1}: {len(valid_ones)} aprovados de {len(batch)} enviados.")
            
        except Exception as e:
            print(f"   Erro no Batch {i}: {e}")

    print(f"Total de novas conexões validadas: {len(validated_edges)}")


 Arquiteto Validando Candidatos...
   Batch 1: 8 aprovados de 16 enviados.
Total de novas conexões validadas: 8


**Agente Pedagogo (Teacher):** Analisa as novas conexões (e as existentes) para identificar dependências de aprendizado críticas, promovendo relações para 'PREREQUISITE' quando um conceito é fundamental para entender o outro.

In [7]:
## 6. Agente 3: The Teacher (Pedagogo)
# Converte relações funcionais (USE/RELATED) em Pedagógicas (PREREQUISITE)

class PrerequisiteCheck(BaseModel):
    indices: List[int] = Field(description="Indices das relações que são Pré-requisitos")

teacher_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Identify LEARNING DEPENDENCIES. If knowing A is mandatory to understand B, select it."),
        ("user", "Relations:\n{items}\n{format_instructions}")
    ])
    | llm_mini
    | PydanticOutputParser(pydantic_object=PrerequisiteCheck)
)

# Converter validated_edges para formato dict compatível
new_relations_dicts = [
    {"source": e.source, "target": e.target, "type": e.relation_type, "explanation": e.explanation} 
    for e in validated_edges
]

# Combinar com antigas (que não eram prereq)
candidates_teacher = new_relations_dicts

if candidates_teacher:
    print("\n🎓 Teacher analisando novas relações...")
    BATCH_SIZE = 50
    upgraded_count = 0
    
    for i in range(0, len(candidates_teacher), BATCH_SIZE):
        batch = candidates_teacher[i:i+BATCH_SIZE]
        batch_fmt = [f"{idx}: {r['source']} -> {r['target']} ({r['type']})" for idx, r in enumerate(batch)]
        
        try:
            res = teacher_chain.invoke({
                "items": "\n".join(batch_fmt),
                "format_instructions": PydanticOutputParser(pydantic_object=PrerequisiteCheck).get_format_instructions()
            })
            
            for rel_idx in res.indices:
                if rel_idx < len(batch):
                    real_idx = i + rel_idx
                    candidates_teacher[real_idx]['type'] = 'PREREQUISITE'
                    candidates_teacher[real_idx]['explanation'] += " [Teacher: PREREQ]"
                    upgraded_count += 1
        except Exception as e:
            print(f"Erro Teacher: {e}")
            
    print(f"Teacher promoveu {upgraded_count} relações para PREREQUISITE.")


🎓 Teacher analisando novas relações...
Teacher promoveu 2 relações para PREREQUISITE.


**Consolidação Final:** Unifica as relações originais com as novas conexões densificadas, recalcula as métricas do grafo (densidade) e salva o resultado final para uso no SINKT.

In [8]:
## 7. Consolidação e Relatório

# Unir Antigas + Novas
final_relations = relations + candidates_teacher

# Construir Grafo Final
G_final = nx.DiGraph()
for c in clean_concepts:
    G_final.add_node(c['nome'])

for r in final_relations:
    if G_final.has_node(r['source']) and G_final.has_node(r['target']):
        G_final.add_edge(r['source'], r['target'], type=r['type'])

old_density = nx.density(G_initial)
new_density = nx.density(G_final)

print("="*40)
print("RELATÓRIO DE DENSIFICAÇÃO AGRESSIVA")
print("="*40)
print(f"Densidade: {old_density:.5f} -> {new_density:.5f}")
print(f"Nós: {len(concepts)} -> {G_final.number_of_nodes()}")
print(f"Arestas: {len(relations)} -> {G_final.number_of_edges()} (+{len(candidates_teacher)})")

output_data = {
    "concepts": clean_concepts,
    "relations": final_relations,
    "metrics": {
        "density": new_density,
        "edges_added": len(candidates_teacher)
    }
}

with open(ENHANCED_FILE, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=4, ensure_ascii=False)
    
print(f"💾 Grafo Denso Salvo: {ENHANCED_FILE}")

RELATÓRIO DE DENSIFICAÇÃO AGRESSIVA
Densidade: 0.00585 -> 0.00435
Nós: 230 -> 204
Arestas: 211 -> 180 (+8)
💾 Grafo Denso Salvo: output/02_densification/enhanced_graph.json
